# 原稿の図作成ノートブック

In [7]:
import numpy as np
import matplotlib
matplotlib.use('Agg')  # GUI を使わないバックエンドに設定
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyArrowPatch
from pathlib import Path
import pickle
from config import method_name_dict, DT, V_MIN, V_MAX, W_MIN, W_MAX, A_MAX, AW_MAX


plt.rcParams["font.size"] = 9  # 全体のフォントサイズが変更されます。
plt.rcParams["font.family"] = "Times New Roman"  # 全体のフォントを設定
plt.rcParams["mathtext.fontset"] = "stix"  # math fontの設定
plt.rcParams["font.weight"] = "normal"  # フォントの太さを細字に設定
plt.rcParams["axes.linewidth"] = 1.0  # axis line width
plt.rcParams["axes.grid"] = True  # make grid
plt.rcParams["legend.edgecolor"] = "black"  # edgeの色を変更
plt.rcParams["legend.handlelength"] = 1  # 凡例の線の長さを調節

In [8]:
from path import step_curves
paths_step = step_curves()
path = paths_step[-1]

# pickleを読み込む
results_path = sorted(Path("/home/decwest/decwest_workspace/dwpp/results").glob("step*"))

# 図1

PP, APP, RPP, DWPPの目標回転速度、実現回転速度を比較した図

一番RMSEの差が大きかった経路7の、回転速度における図を書く


In [9]:
path = paths_step[7]

# pickleを読み込む
result_path = results_path[7]
# robot posesの読み込み
robot_poses_path = result_path / "robot_poses_dict.pkl"
with open(robot_poses_path, "rb") as f:
    robot_poses_dist = pickle.load(f)
# break_constraints_flagsの読み込み
break_constraints_flags_path = result_path / "break_constraints_flags_dict.pkl"
with open(break_constraints_flags_path, "rb") as f:
    break_constraints_flags_dict = pickle.load(f)
# robot_velocities_dictの読み込み
robot_velocities_dict_path = result_path / "robot_velocities_dict.pkl"
with open(robot_velocities_dict_path, "rb") as f:
    robot_velocities_dict = pickle.load(f)
# robot_velocities_dictの読み込み
robot_ref_velocities_dict_path = result_path / "robot_ref_velocities_dict.pkl"
with open(robot_ref_velocities_dict_path, "rb") as f:
    robot_ref_velocities_dict = pickle.load(f)
# timestampsの読み込み
time_stamp_dict_path = result_path / "time_stamp_dict.pkl"
with open(time_stamp_dict_path, "rb") as f:
    time_stamp_dict = pickle.load(f)

In [10]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 2))
ax1 = fig.add_subplot(141)
ax2 = fig.add_subplot(142)
ax3 = fig.add_subplot(143)
ax4 = fig.add_subplot(144)

def plot_velocity_profile(ax, robot_velocities, robot_ref_velocities, time_stamps, title, add_legend=False):
    time_stamps = np.array(time_stamps)
    robot_ref_velocities = np.array(robot_ref_velocities)
    robot_velocities = np.array(robot_velocities)

    # 速度の上限・下限線
    ax.axhline(y=W_MAX, linestyle="--", c="black")
    ax.axhline(y=W_MIN, linestyle="--", c="black")
    
    # 凡例のラベルを付けるのは最初のプロットだけ
    if add_legend:
        ax.plot(time_stamps, robot_ref_velocities[:, 1], label='$\omega_{ref}$', c="blue")
        ax.plot(time_stamps, robot_velocities[:, 1], label='$\omega$', c="red")
    else:
        ax.plot(time_stamps, robot_ref_velocities[:, 1], c="blue")
        ax.plot(time_stamps, robot_velocities[:, 1], c="red")

    ax.set_title(title)
    ax.set_xlabel('time [s]')
    if title == "Pure Pursuit":
        ax.set_ylabel('$\omega$ [rad/s]')
    ax.set_xlim(0, 21.0)
    ax.set_ylim(W_MIN-0.2, W_MAX+0.2)

# 最初のプロット(ax1) のみ凡例ラベルを設定
plot_velocity_profile(ax1, robot_velocities_dict["pp"], robot_ref_velocities_dict["pp"], time_stamp_dict["pp"], "Pure Pursuit", add_legend=True)
plot_velocity_profile(ax2, robot_velocities_dict["app"], robot_ref_velocities_dict["app"], time_stamp_dict["app"], "Adaptive")
plot_velocity_profile(ax3, robot_velocities_dict["rpp"], robot_ref_velocities_dict["rpp"], time_stamp_dict["rpp"], "Regulated")
plot_velocity_profile(ax4, robot_velocities_dict["dwpp"], robot_ref_velocities_dict["dwpp"], time_stamp_dict["dwpp"], "Dynamic Window")

# `ax1` の凡例情報を取得し、fig.legend() に渡す
# handles, labels = ax1.get_legend_handles_labels()
# fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.15), ncol=2)

plt.tight_layout()

# グラフを保存
plt.savefig(f'../results/figure1.svg')


ROBOMECH用

In [11]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(4, 4))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

def plot_velocity_profile(ax, robot_velocities, robot_ref_velocities, time_stamps, title, add_legend=False):
    time_stamps = np.array(time_stamps)
    robot_ref_velocities = np.array(robot_ref_velocities)
    robot_velocities = np.array(robot_velocities)

    # 速度の上限・下限線
    ax.axhline(y=W_MAX, linestyle="--", c="black")
    ax.axhline(y=W_MIN, linestyle="--", c="black")
    
    # 凡例のラベルを付けるのは最初のプロットだけ
    if add_legend:
        ax.plot(time_stamps, robot_ref_velocities[:, 1], label='$\omega_{ref}$', c="blue")
        ax.plot(time_stamps, robot_velocities[:, 1], label='$\omega$', c="red")
    else:
        ax.plot(time_stamps, robot_ref_velocities[:, 1], c="blue")
        ax.plot(time_stamps, robot_velocities[:, 1], c="red")

    ax.set_title(title)
    ax.set_xlabel('time [s]')
    if title == "PP" or title == "RPP" :
        ax.set_ylabel('$\omega$ [rad/s]')
    ax.set_xlim(0, 21.0)
    ax.set_ylim(W_MIN-0.2, W_MAX+0.2)

# 最初のプロット(ax1) のみ凡例ラベルを設定
plot_velocity_profile(ax1, robot_velocities_dict["pp"], robot_ref_velocities_dict["pp"], time_stamp_dict["pp"], "PP", add_legend=True)
plot_velocity_profile(ax2, robot_velocities_dict["app"], robot_ref_velocities_dict["app"], time_stamp_dict["app"], "APP")
plot_velocity_profile(ax3, robot_velocities_dict["rpp"], robot_ref_velocities_dict["rpp"], time_stamp_dict["rpp"], "RPP")
plot_velocity_profile(ax4, robot_velocities_dict["dwpp"], robot_ref_velocities_dict["dwpp"], time_stamp_dict["dwpp"], "DWPP (proposed)")

# `ax1` の凡例情報を取得し、fig.legend() に渡す
# handles, labels = ax1.get_legend_handles_labels()
# fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.15), ncol=2)

plt.tight_layout()

# グラフを保存
plt.savefig(f'../results/figure1_robomech.svg')


# 図4

vω空間におけるRPP, DWPPの目標速度決定の違い

ここはスライドで作る。実現された結果は、図7で示す。

# 図5

経路の外観図を書く

In [12]:
path = paths_step[7]
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot(111)
ax.set_xlim(-1.2213203435596425, 1.9786796564403577)
ax.set_ylim(-0.1, 2.5)

# ax.set_xlabel('$x$ [m]')
# ax.set_ylabel('$y$ [m]')
ax.set_aspect('equal')

# パス
ax.plot(path[:, 0], path[:, 1], 'k--', label='Path')

plt.tight_layout()

# グラフを表示
plt.savefig(f'../results/path_example.png')

# 図6

最もRMSEで差がついた参照経路における各種法の追従経路

制約違反したところはハイライトする

## 最もRMSEで差がついた場所

In [13]:
for result_path in results_path:
    print(result_path)
    rmse_path = result_path / "rmse_dict.pkl"
    with open(rmse_path, "rb") as f:
        results = pickle.load(f)
    for key in results:
        print(key, results[key])

/home/decwest/decwest_workspace/dwpp/results/step_curve_0
pp 0.019133561910436584
app 0.02033568924789376
rpp 0.020433731871953593
dwpp 0.020211633003320243
/home/decwest/decwest_workspace/dwpp/results/step_curve_1
pp 0.01819043386174176
app 0.019572215834299782
rpp 0.019632124894792657
dwpp 0.019204306983573137
/home/decwest/decwest_workspace/dwpp/results/step_curve_2
pp 0.018631241930987846
app 0.01965391343711651
rpp 0.01961759517473724
dwpp 0.019420072921760533
/home/decwest/decwest_workspace/dwpp/results/step_curve_3
pp 0.11647772816323401
app 0.10054840484383912
rpp 0.03392521301690285
dwpp 0.03464689233143726
/home/decwest/decwest_workspace/dwpp/results/step_curve_4
pp 0.15272137618747889
app 0.1030939018709039
rpp 0.03285514120437314
dwpp 0.03287012998713747
/home/decwest/decwest_workspace/dwpp/results/step_curve_5
pp 0.14448296961894402
app 0.09909577871802867
rpp 0.0317456881457999
dwpp 0.031240790851314607
/home/decwest/decwest_workspace/dwpp/results/step_curve_6
pp 0.376175

一番RMSEが小さいのは、経路7

In [14]:
results_path

[PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_0'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_1'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_2'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_3'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_4'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_5'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_6'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_7'),
 PosixPath('/home/decwest/decwest_workspace/dwpp/results/step_curve_8')]

In [15]:
path = paths_step[7]

# pickleを読み込む
result_path = results_path[7]
# robot posesの読み込み
robot_poses_path = result_path / "robot_poses_dict.pkl"
with open(robot_poses_path, "rb") as f:
    robot_poses_dist = pickle.load(f)
# break_constraints_flagsの読み込み
break_constraints_flags_path = result_path / "break_constraints_flags_dict.pkl"
with open(break_constraints_flags_path, "rb") as f:
    break_constraints_flags_dict = pickle.load(f)

fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot(111)
ax.set_xlim(np.min(path[:, 0]) - 0.1, np.max(path[:, 0]) + 0.1)
ax.set_ylim(np.min(path[:, 1]) - 0.1, np.max(path[:, 1]) + 0.8)
print(np.min(path[:, 0]) - 0.1, np.max(path[:, 0]) + 0.1)
print(np.min(path[:, 1]) - 0.1, np.max(path[:, 1]) + 1)
# ax.set_xlabel('$x$ [m]')
# ax.set_ylabel('$y$ [m]')
ax.set_aspect('equal')

# パス
ax.plot(path[:, 0], path[:, 1], 'k--', label='Reference Path')


# 制約違反した場所をハイライト表示
for method_name, break_constraints_flags in break_constraints_flags_dict.items():
    break_constraints_flags = np.array(break_constraints_flags)
    
    for i, flag in enumerate(break_constraints_flags):
        if flag[0] and flag[1]: # 並進回転共に違反
            x = robot_poses_dist[method_name][i][0]
            y = robot_poses_dist[method_name][i][1]
            ax.plot(
                x,
                y,
                marker='o',
                markersize=4,
                linestyle='None',     # 線でつなげない
                color='yellow',
                alpha=0.3             # 半透明（0〜1の値が指定可）
            )
        elif flag[0]: # 並進だけ違反
            x = robot_poses_dist[method_name][i][0]
            y = robot_poses_dist[method_name][i][1]
            ax.plot(
                x,
                y,
                marker='o',
                markersize=4,
                linestyle='None',     # 線でつなげない
                color='red',
                alpha=0.3             # 半透明（0〜1の値が指定可）
            )
        elif flag[1]: # 回転だけ違反
            x = robot_poses_dist[method_name][i][0]
            y = robot_poses_dist[method_name][i][1]
            ax.plot(
                x,
                y,
                marker='o',
                markersize=4,
                linestyle='None',     # 線でつなげない
                color='blue',
                alpha=0.3             # 半透明（0〜1の値が指定可）
            )


# ロボットの位置
for method_name, robot_poses in robot_poses_dist.items():
    robot_poses = np.array(robot_poses)
    ax.plot(robot_poses[:, 0], robot_poses[:, 1], label=method_name_dict[method_name])

# 凡例を表示
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.legend()

plt.tight_layout()

# グラフを表示
plt.savefig(f'../results/tracking_result_example.png')

-1.2213203435596425 1.9786796564403577
-0.1 3.121320343559643


# 図7

最もRMSEで差がついたl=3.0, θ=135°の参照経路におけるRPP, DWPP with RPPのvω空間におけるDynamic windowと目標速度の遷移を図7に図示する。 

- 速度プロファイル
  - 走行位置のズレが少ない曲がり始めに着目してプロットする（完了）

Dynamic Window付近を切り取って可視化するようにする。

- vreg>=vminの好例（並進速度が減速することを示す）
  - 経路7のRPP, DWPPの31番目を同時にプロットする！！
- vreg<=vminの好例（回転速度が増加することを示す）
  - DWPPの36番目と、RPPの47番目あたりはいい対比になる。

速度プロファイルを比較する

In [16]:
path = paths_step[7]

# pickleを読み込む
result_path = results_path[7]
# robot posesの読み込み
robot_poses_path = result_path / "robot_poses_dict.pkl"
with open(robot_poses_path, "rb") as f:
    robot_poses_dist = pickle.load(f)
# break_constraints_flagsの読み込み
break_constraints_flags_path = result_path / "break_constraints_flags_dict.pkl"
with open(break_constraints_flags_path, "rb") as f:
    break_constraints_flags_dict = pickle.load(f)
# robot_velocities_dictの読み込み
robot_velocities_dict_path = result_path / "robot_velocities_dict.pkl"
with open(robot_velocities_dict_path, "rb") as f:
    robot_velocities_dict = pickle.load(f)
# timestampsの読み込み
time_stamp_dict_path = result_path / "time_stamp_dict.pkl"
with open(time_stamp_dict_path, "rb") as f:
    time_stamp_dict = pickle.load(f)

並進速度

In [17]:
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)
ax.set_xlabel('$t$ [s]')
ax.set_ylabel('$v$ [m/s]')
# 速度の上限・下限線
ax.axhline(y=V_MAX, linestyle="--", c="black")
ax.axhline(y=V_MIN, linestyle="--", c="black")

rpp_timestamp = np.array(time_stamp_dict["rpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_timestamp = np.array(time_stamp_dict["dwpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])

# 並進目標速度
ax.plot(rpp_timestamp, rpp_velocities[:, 0], c="blue", label='RPP')
# 並進実現速度
ax.plot(dwpp_timestamp, dwpp_velocities[:, 0], c="red", label='DWPP')

# x軸の値の範囲を指定
ax.set_xlim(0, 23.0)

# 凡例を表示
plt.legend(loc='upper right', bbox_to_anchor=(0.4, -0.15), ncol=2)

# ax1.legend()
plt.tight_layout()


# グラフを表示
plt.savefig(f'../results/figure7_trans.png')


回転速度

In [18]:
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)
ax.set_xlabel('$t$ [s]')
ax.set_ylabel('$v$ [m/s]')
# 速度の上限・下限線
ax.axhline(y=W_MAX, linestyle="--", c="black")
ax.axhline(y=W_MIN, linestyle="--", c="black")

rpp_timestamp = np.array(time_stamp_dict["rpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_timestamp = np.array(time_stamp_dict["dwpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])

# 並進目標速度
ax.plot(rpp_timestamp, rpp_velocities[:, 1], c="blue", label='RPP')
# 並進実現速度
ax.plot(dwpp_timestamp, dwpp_velocities[:, 1], c="red", label='DWPP')

# x軸の値の範囲を指定
ax.set_xlim(0, 23.0)

# 凡例を表示
plt.legend(loc='upper right', bbox_to_anchor=(0.4, -0.15), ncol=2)

# ax1.legend()
plt.tight_layout()


# グラフを表示
plt.savefig(f'../results/figure7_rot.png')


並進・回転を一枚で書く

In [52]:
plt.rcParams["font.size"] = 54  # 全体のフォントサイズが変更されます。
plt.rcParams["font.family"] = "Times New Roman"  # 全体のフォントを設定
plt.rcParams["mathtext.fontset"] = "stix"  # math fontの設定
plt.rcParams["font.weight"] = "normal"  # フォントの太さを細字に設定
plt.rcParams["axes.linewidth"] = 1.0  # axis line width
plt.rcParams["axes.grid"] = True  # make grid
plt.rcParams["legend.edgecolor"] = "black"  # edgeの色を変更
plt.rcParams["legend.handlelength"] = 1  # 凡例の線の長さを調節

rpp_timestamp = np.array(time_stamp_dict["rpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_timestamp = np.array(time_stamp_dict["dwpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])

fig = plt.figure(figsize=(18, 8))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('$t$ [s]')
ax1.set_ylabel('$v$ [m/s]')
# 速度の上限・下限線
ax1.axhline(y=V_MAX, linestyle="--", c="black")
ax1.axhline(y=V_MIN, linestyle="--", c="black")

# 並進目標速度
ax1.plot(rpp_timestamp, rpp_velocities[:, 0], c="blue", label='RPP', linewidth=6)
# 並進実現速度
ax1.plot(dwpp_timestamp, dwpp_velocities[:, 0], c="red", label='DWPP', linewidth=6)

# x軸の値の範囲を指定
ax1.set_xlim(0, 3.0)

ax1.set_yticks([0.0, 0.25, 0.50])
ax1.set_yticklabels(['0.0', '0.25', '0.50'])  # 数式で表示

ax1.set_xticks([0.0, 1.0, 1.5, 2.0, 2.5])
ax1.set_xticklabels(['0.0', '1.0', '1.5', '2.0', '2.5'])  # 数式で表示

ax2 = fig.add_subplot(122)
ax2.set_xlabel('$t$ [s]')
ax2.set_ylabel('$\omega$ [m/s]')
# 速度の上限・下限線
ax2.axhline(y=W_MAX, linestyle="--", c="black")
ax2.axhline(y=W_MIN, linestyle="--", c="black")

# 並進目標速度
ax2.plot(rpp_timestamp, rpp_velocities[:, 1], c="blue", label='RPP', linewidth=6)
# 並進実現速度
ax2.plot(dwpp_timestamp, dwpp_velocities[:, 1], c="red", label='DWPP', linewidth=6)

# x軸の値の範囲を指定
ax2.set_xlim(0, 3.0)

ax2.set_yticks([-1.0, -0.5, 0.0, 0.5, 1.0])
ax2.set_yticklabels(['-1.0', '-0.5', '0.0', '0.5', '1.0'])  # 数式で表示
ax2.set_xticks([0.0, 1.0, 1.5, 2.0, 2.5])
ax2.set_xticklabels(['0.0', '1.0', '1.5', '2.0', '2.5'])  # 数式で表示

# 凡例を表示
# plt.legend(loc='upper right', bbox_to_anchor=(0.4, -0.3), ncol=2)
# plt.legend()

# ax1.legend()
plt.tight_layout()


# グラフを表示
plt.savefig(f'../results/figure7.svg')


Dynamic Windowの図示

In [20]:
result_path = results_path[7]
# pickleを読み込む
# curvatureの読み込み
curvature_dict_path = result_path / "curvatures_flags_dict.pkl"
with open(curvature_dict_path, "rb") as f:
    curvature_dict = pickle.load(f)
# regulated_vの読み込み
regulated_v_path = result_path / "regulated_vs_flags_dict.pkl"
with open(regulated_v_path, "rb") as f:
    regulated_v = pickle.load(f)
# robot velocityの読み込み
robot_velocities_dict_path = result_path / "robot_velocities_dict.pkl"
with open(robot_velocities_dict_path, "rb") as f:
    robot_velocities_dict = pickle.load(f)
# robot ref velocityの読み込み
robot_ref_velocities_dict_path = result_path / "robot_ref_velocities_dict.pkl"
with open(robot_ref_velocities_dict_path, "rb") as f:
    robot_ref_velocities_dict = pickle.load(f)


rpp_curvatures = np.array(curvature_dict["rpp"])
dwpp_curvatures = np.array(curvature_dict["dwpp"])
rpp_regulated_v = np.array(regulated_v["rpp"])
dwpp_regulated_v = np.array(regulated_v["dwpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])
rpp_ref_velocities = np.array(robot_ref_velocities_dict["rpp"])
dwpp_ref_velocities = np.array(robot_ref_velocities_dict["dwpp"])


In [21]:
def plot_vw_plane(curvatures, regulated_vs, ref_velocitys, velocitys, plot_range, method_name, xmin, xmax, ymin, ymax):
    for idx, (curvature, regulated_v, ref_velocity, velocity) in enumerate(zip(curvatures, regulated_vs, ref_velocitys, velocitys)):
        if idx in plot_range:
            fig = plt.figure(figsize=(2, 2))
            ax = fig.add_subplot(111)
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_xlabel('$v$')
            ax.set_ylabel('$\omega$')
            ax.set_title(f'$t = {idx*0.05:.2f}$')
            
            # 速度の上限・下限線
            ax.axvline(x=V_MAX, linestyle="--", c="black")
            ax.axvline(x=V_MIN, linestyle="--", c="black")
            ax.axhline(y=W_MIN, linestyle="--", c="black")
            ax.axhline(y=W_MAX, linestyle="--", c="black")
            
            # regulatedされた上限値
            ax.axvline(x=regulated_v, linestyle="--", c="red", label="regulated v")
            
            # dynamic window の計算
            dw_vmax = min(velocitys[idx-1][0] + A_MAX * DT, V_MAX)
            dw_vmin = max(velocitys[idx-1][0] - A_MAX * DT, V_MIN)
            dw_wmax = min(velocitys[idx-1][1] + AW_MAX * DT, W_MAX)
            dw_wmin = max(velocitys[idx-1][1] - AW_MAX * DT, W_MIN)
            if dw_vmax > regulated_v:
                dw_vmax = max(dw_vmin, regulated_v)
            
            ax.plot([dw_vmin, dw_vmin], [dw_wmin, dw_wmax], c="black", label="dynamic window")
            ax.plot([dw_vmax, dw_vmax], [dw_wmin, dw_wmax], c="black")
            ax.plot([dw_vmin, dw_vmax], [dw_wmin, dw_wmin], c="black")
            ax.plot([dw_vmin, dw_vmax], [dw_wmax, dw_wmax], c="black")
            
            # 曲率に対応する直線
            ax.axline((0, 0), slope=curvature, color='red', lw=1, label="$\omega={\phi}v$")
            
            # 参照速度, 現在速度, 一時刻前の速度
            ax.scatter(ref_velocity[0], ref_velocity[1], c="green", label="reference velocity", s=15)
            ax.scatter(velocity[0], velocity[1], c="red", label="next velocity", s=15)
            ax.scatter(velocitys[idx-1][0], velocitys[idx-1][1], c="black", label="current velocity", s=15)
            
            # ax.legend()
            ax.set_aspect('equal')
            plt.tight_layout()
            # グラフを表示
            plt.savefig(f'../results/figure7/{method_name}_{idx}.svg')
            
plot_vw_plane(rpp_curvatures, rpp_regulated_v, rpp_ref_velocities, rpp_velocities, range(30, 36), "rpp",\
xmin=0.4, xmax=V_MAX+0.05, ymin=-0.05, ymax=0.3)
plot_vw_plane(dwpp_curvatures, dwpp_regulated_v, dwpp_ref_velocities, dwpp_velocities, range(30, 36), "dwpp",\
xmin=0.4, xmax=V_MAX+0.05, ymin=-0.05, ymax=0.3)


In [22]:
plot_vw_plane(rpp_curvatures, rpp_regulated_v, rpp_ref_velocities, rpp_velocities, range(40, 46), "rpp",\
xmin=0.00, xmax=0.35, ymin=0.4, ymax=1.05)
plot_vw_plane(dwpp_curvatures, dwpp_regulated_v, dwpp_ref_velocities, dwpp_velocities, range(40, 46), "dwpp",\
xmin=0.00, xmax=0.35, ymin=0.4, ymax=1.05)


/tmp/ipykernel_7300/3776434733.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(2, 2))


# Table 1

RMSEの表を書く

In [23]:
import pandas as pd

# データフレームを作成
indexes = ["2.0", "3.0", "4.0"]
columns = ["45", "90", "135"]
pp_rmse_df = pd.DataFrame(index=indexes, columns=columns)
app_rmse_df = pd.DataFrame(index=indexes, columns=columns)
rpp_rmse_df = pd.DataFrame(index=indexes, columns=columns)
dwpp_rmse_df = pd.DataFrame(index=indexes, columns=columns)

for idx, result_path in enumerate(results_path):
    # pickleを読み込む
    # rmseの読み込み
    rmse_dict_path = result_path / "rmse_dict.pkl"
    with open(rmse_dict_path, "rb") as f:
        rmse_dict = pickle.load(f)
    
    if idx // 3 == 0:
        c = "45"
    elif idx // 3 == 1:
        c = "90"
    elif idx // 3 == 2:
        c = "135"
    
    if idx % 3 == 0:
        l = "2.0"
    elif idx % 3 == 1:
        l = "3.0"
    elif idx % 3 == 2:
        l = "4.0"
    
    pp_rmse_df.loc[l, c] = rmse_dict["pp"]
    app_rmse_df.loc[l, c] = rmse_dict["app"]
    rpp_rmse_df.loc[l, c] = rmse_dict["rpp"]
    dwpp_rmse_df.loc[l, c] = rmse_dict["dwpp"]

    
def calc_mean_and_var(df: pd.DataFrame):
    # 各行の平均・標準偏差を計算（元の列だけで計算）
    row_means = df.mean(axis=1)
    row_stds = df.std(axis=1, ddof=0)

    # 各列の平均・標準偏差を計算（元の行だけで計算）
    col_means = df.mean(axis=0)
    col_stds = df.std(axis=0, ddof=0)
    
    # 全体のmeanとstdを算出
    mean = np.mean(df.values)
    std = np.std(df.values)
    
    # 行方向の集計を新しい列として追加
    df['RowMean'] = row_means
    df['RowStd'] = row_stds
    
    # 列方向の集計を新しい行として追加
    # 追加する行には元の列分の値に加えて、RowMean/RowVar 列用に None (または NaN) を入れる
    df.loc['ColMean'] = list(col_means) + [mean, None]
    df.loc['ColStd']  = list(col_stds)  + [None, std]
    
    

calc_mean_and_var(pp_rmse_df)
calc_mean_and_var(app_rmse_df)
calc_mean_and_var(rpp_rmse_df)
calc_mean_and_var(dwpp_rmse_df)
    
pp_rmse_df = pp_rmse_df.astype(float)
app_rmse_df = app_rmse_df.astype(float)
rpp_rmse_df = rpp_rmse_df.astype(float)
dwpp_rmse_df = dwpp_rmse_df.astype(float)

In [24]:
pp_rmse_df

,45,90,135,RowMean,RowStd
2.0,0.019134,0.116478,0.376176,0.170596,0.150701
3.0,0.018190,0.152721,0.365168,0.178693,0.142838
4.0,0.018631,0.144483,0.354796,0.172637,0.138675
ColMean,0.018652,0.137894,0.365380,0.173975,NaN
ColStd,0.000385,0.015513,0.008729,NaN,0.144199


In [25]:
app_rmse_df

,45,90,135,RowMean,RowStd
2.0,0.020336,0.100548,0.378233,0.166372,0.153345
3.0,0.019572,0.103094,0.333141,0.151936,0.132591
4.0,0.019654,0.099096,0.322227,0.146992,0.128084
ColMean,0.019854,0.100913,0.344534,0.155100,NaN
ColStd,0.000342,0.001652,0.024242,NaN,0.138688


In [26]:
rpp_rmse_df

,45,90,135,RowMean,RowStd
2.0,0.020434,0.033925,0.072778,0.042379,0.022190
3.0,0.019632,0.032855,0.092879,0.048455,0.031872
4.0,0.019618,0.031746,0.088468,0.046610,0.030009
ColMean,0.019894,0.032842,0.084708,0.045815,NaN
ColStd,0.000381,0.000890,0.008626,NaN,0.028450


In [27]:
dwpp_rmse_df

,45,90,135,RowMean,RowStd
2.0,0.020212,0.034647,0.064972,0.039943,0.018653
3.0,0.019204,0.032870,0.059383,0.037152,0.016680
4.0,0.019420,0.031241,0.056192,0.035618,0.015328
ColMean,0.019612,0.032919,0.060182,0.037571,NaN
ColStd,0.000433,0.001391,0.003629,NaN,0.017036


# Table 2

並進・回転方向それぞれの違反率

In [28]:
from collections import defaultdict
break_ratio_dict = defaultdict(list)

for idx, result_path in enumerate(results_path):
    
    # l=3.0の結果のみ取り出す
    if idx % 3 == 0:
        # l = "2.0"
        continue
    elif idx % 3 == 1:
        # l = "3.0"
        pass
    elif idx % 3 == 2:
        # l = "4.0"
        continue
    
    
    # pickleを読み込む
    # 制約違反フラグの読み込み
    break_constraints_flags_dict_path = result_path / "break_constraints_flags_dict.pkl"
    with open(break_constraints_flags_dict_path, "rb") as f:
        break_constraints_flags_dict = pickle.load(f)
    
    def calc_trans_and_rot_break_constraints(break_constraints_flags):
        break_constraints_flags = np.array(break_constraints_flags)
        trans_break_constraints_flags = break_constraints_flags[:, 0]
        rot_break_constraints_flags = break_constraints_flags[:, 1]
        
        trans_break_ratio = sum(trans_break_constraints_flags) / len(trans_break_constraints_flags) * 100.0
        rot_break_ratio = sum(rot_break_constraints_flags) / len(rot_break_constraints_flags) * 100.0
        
        return trans_break_ratio, rot_break_ratio
    
    pp_trans_break_ratio, pp_rot_break_ratio = calc_trans_and_rot_break_constraints(break_constraints_flags_dict["pp"])
    app_trans_break_ratio, app_rot_break_ratio = calc_trans_and_rot_break_constraints(break_constraints_flags_dict["app"])
    rpp_trans_break_ratio, rpp_rot_break_ratio = calc_trans_and_rot_break_constraints(break_constraints_flags_dict["rpp"])
    dwpp_trans_break_ratio, dwpp_rot_break_ratio = calc_trans_and_rot_break_constraints(break_constraints_flags_dict["dwpp"])
    
    break_ratio_dict["pp_trans"].append(pp_trans_break_ratio)
    break_ratio_dict["pp_rot"].append(pp_rot_break_ratio)
    break_ratio_dict["app_trans"].append(app_trans_break_ratio)
    break_ratio_dict["app_rot"].append(app_rot_break_ratio)
    break_ratio_dict["rpp_trans"].append(rpp_trans_break_ratio)
    break_ratio_dict["rpp_rot"].append(rpp_rot_break_ratio)
    break_ratio_dict["dwpp_trans"].append(dwpp_trans_break_ratio)
    break_ratio_dict["dwpp_rot"].append(dwpp_rot_break_ratio)
    
for key in break_ratio_dict:
    break_ratio_dict[key] = np.array(break_ratio_dict[key])
    mean = np.mean(break_ratio_dict[key])
    std = np.std(break_ratio_dict[key])
    print(key, mean, std)

pp_trans 11.625579907775794 0.9086983798802273
pp_rot 39.32093363065982 19.4398714781478
app_trans 12.110782129740533 0.7425293478889028
app_rot 26.040249231952146 13.659926956119161
rpp_trans 19.91883625674113 5.088206080112798
rpp_rot 10.239492483437624 2.6461266195880437
dwpp_trans 0.0 0.0
dwpp_rot 0.0 0.0


In [29]:
len(break_ratio_dict["pp_trans"])

3